## Limpeza de dados Cartola ano 2015

In [26]:
from pyspark.sql import HiveContext
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import lower, col, lit, regexp_replace
from pyspark.sql.functions import lit
import pandas as pd
import json
import requests

## Jogadores

In [27]:
#Carrega arquivo CSV
jogadores_2015_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2015/2015_jogadores.csv", header=True)
sorted_jogadores_2015_df = jogadores_2015_df.sort(jogadores_2015_df.ID.desc())


In [28]:
#Adiciona coluna ANO = 2015
#Clube ID está como NA????
jogadores_2015_ano = sorted_jogadores_2015_df.withColumn('ano', lit(2015))
jogadores_2015_ano.show()

+-----+-----------------+-------+---------+----+
|   ID|          Apelido|ClubeID|PosicaoID| ano|
+-----+-----------------+-------+---------+----+
|93378|          Augusto|     NA|        3|2015|
|93377|           Jobson|     NA|        4|2015|
|93376|     Wesley Souza|     NA|        4|2015|
|93368|  Lucas Veríssimo|     NA|        3|2015|
|93317|  Diogo Giacomini|     NA|        6|2015|
|93283|          Rafinha|     NA|        4|2015|
|93282|            Fabio|     NA|        4|2015|
|93281|          Adriano|     NA|        5|2015|
|93153|      Pachequinho|     NA|        6|2015|
|93152|          Tontini|     NA|        4|2015|
|93151|    Thiago Santos|     NA|        5|2015|
|93108|     Mateus Vital|     NA|        4|2015|
|93028|     Danny Sergio|     NA|        6|2015|
|92981|          Ronaldo|     NA|        4|2015|
|92901|   Felipe Moreira|     NA|        6|2015|
|92848|     Breno Santos|     NA|        2|2015|
|92841|Fernando Medeiros|     NA|        4|2015|
|92837|    Sergio Vi

In [29]:
jogadores_2015_ano.write.parquet('/cartola/clean/jogadores/2015')

In [30]:
#transforma arquivo em parquet
jogadores_2015_parquet = spark.read.parquet('/cartola/clean/jogadores/*')

In [31]:
jogadores_2015_parquet.show()
jogadores_2015_parquet.schema


+-----+--------------------+-------+---------+----+
|   ID|             Apelido|ClubeID|PosicaoID| ano|
+-----+--------------------+-------+---------+----+
|51683|        Bruno Rangel|    315|        5|2014|
|51705|       Bruno Rodrigo|    283|        3|2014|
|51772|     Éverton Ribeiro|    283|        4|2014|
|51779|       Pedro Botelho|    282|        2|2014|
|51781|               Ávine|    265|        2|2014|
|51985|      Anderson Pedra|    292|        4|2014|
|41387|       Eduardo Costa|    314|        4|2015|
|41327|Vanderlei Luxemburgo|    262|        6|2015|
|41258|     Adilson Batista|     NA|        6|2015|
|41218|              Magrão|    292|        1|2015|
|41159|         João Carlos|     NA|        3|2015|
|68945|              Saimon|     NA|        3|2015|
|68938|    Giovanni Augusto|    282|        4|2015|
|68932|       Renan Ribeiro|    276|        1|2015|
|68929|      Thiago Ribeiro|    282|        5|2015|
|68928|              Rafael|    283|        1|2015|
|68925|     

StructType(List(StructField(ID,StringType,true),StructField(Apelido,StringType,true),StructField(ClubeID,StringType,true),StructField(PosicaoID,StringType,true),StructField(ano,IntegerType,true)))

In [32]:
#Alterar ID para int.(está string)
jogadores_2015_parquet = jogadores_2015_parquet.withColumn("ID", jogadores_2015_parquet["ID"].cast(IntegerType()))
jogadores_2015_parquet = jogadores_2015_parquet.withColumn("ClubeID", jogadores_2015_parquet["ClubeID"].cast(IntegerType()))
jogadores_2015_parquet = jogadores_2015_parquet.withColumn("PosicaoID", jogadores_2015_parquet["PosicaoID"].cast(IntegerType()))

In [33]:
jogadores_2015_parquet.schema

StructType(List(StructField(ID,IntegerType,true),StructField(Apelido,StringType,true),StructField(ClubeID,IntegerType,true),StructField(PosicaoID,IntegerType,true),StructField(ano,IntegerType,true)))

In [34]:
jogadores_2015_parquet.show()

print(jogadores_2015_parquet.count())

+-----+--------------------+-------+---------+----+
|   ID|             Apelido|ClubeID|PosicaoID| ano|
+-----+--------------------+-------+---------+----+
|51683|        Bruno Rangel|    315|        5|2014|
|51705|       Bruno Rodrigo|    283|        3|2014|
|51772|     Éverton Ribeiro|    283|        4|2014|
|51779|       Pedro Botelho|    282|        2|2014|
|51781|               Ávine|    265|        2|2014|
|51985|      Anderson Pedra|    292|        4|2014|
|41387|       Eduardo Costa|    314|        4|2015|
|41327|Vanderlei Luxemburgo|    262|        6|2015|
|41258|     Adilson Batista|   null|        6|2015|
|41218|              Magrão|    292|        1|2015|
|41159|         João Carlos|   null|        3|2015|
|68945|              Saimon|   null|        3|2015|
|68938|    Giovanni Augusto|    282|        4|2015|
|68932|       Renan Ribeiro|    276|        1|2015|
|68929|      Thiago Ribeiro|    282|        5|2015|
|68928|              Rafael|    283|        1|2015|
|68925|     

## Partidas_IDS (temos partidas como NA)

In [108]:
partidas_ids_2015_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2015/2015_partidas_ids.csv", header=True)

In [109]:
partidas_ids_2015_ano = partidas_ids_2015_df.withColumn('ano', lit(2015))

In [110]:
partidas_ids_2015_ano.show(100)

+------+------+------+-----------+----------+---------------+---------+----+
|    ID|Rodada|CasaID|VisitanteID|PlacarCasa|PlacarVisitante|Resultado| ano|
+------+------+------+-----------+----------+---------------+---------+----+
|190053|     1|   276|        262|         2|              1|     Casa|2015|
|190054|     1|   275|        282|         2|              2|   Empate|2015|
|190055|     1|   266|        317|         1|              0|     Casa|2015|
|190056|     1|   267|        290|         0|              0|   Empate|2015|
|190057|     1|   284|        303|         3|              3|   Empate|2015|
|190058|     1|   283|        264|         0|              1|Visitante|2015|
|190059|     1|   314|        277|         1|              1|   Empate|2015|
|190060|     1|   293|        285|         3|              0|     Casa|2015|
|190061|     1|   292|        316|         4|              1|     Casa|2015|
|190062|     1|   315|        294|         2|              1|     Casa|2015|

In [111]:
partidas_ids_2015_ano.write.parquet('/cartola/clean/partidas_ids/2015')

In [112]:
partidas_ids_2015_parquet = spark.read.parquet('/cartola/clean/partidas_ids/*')

In [113]:
partidas_ids_2015_parquet.show(100)
partidas_ids_2015_parquet.schema

+------+------+------+-----------+----------+---------------+---------+----+
|    ID|Rodada|CasaID|VisitanteID|PlacarCasa|PlacarVisitante|Resultado| ano|
+------+------+------+-----------+----------+---------------+---------+----+
|179872|     1|   262|        290|         0|              0|   Empate|2014|
|179873|     1|   266|        316|         3|              0|     Casa|2014|
|179874|     1|   276|        263|         3|              0|     Casa|2014|
|179875|     1|   277|        292|         1|              1|   Empate|2014|
|179876|     1|   293|        284|         1|              0|     Casa|2014|
|179877|     1|   282|        264|         0|              0|   Empate|2014|
|179878|     1|   265|        283|         1|              2|Visitante|2014|
|179879|     1|   285|        287|         1|              0|     Casa|2014|
|179880|     1|   288|        275|         1|              2|Visitante|2014|
|179881|     1|   315|        294|         0|              0|   Empate|2014|

StructType(List(StructField(ID,StringType,true),StructField(Rodada,StringType,true),StructField(CasaID,StringType,true),StructField(VisitanteID,StringType,true),StructField(PlacarCasa,StringType,true),StructField(PlacarVisitante,StringType,true),StructField(Resultado,StringType,true),StructField(ano,IntegerType,true)))

In [115]:
partidas_ids_2015_parquet = partidas_ids_2015_parquet.withColumn("ID", partidas_ids_2015_parquet["ID"].cast(IntegerType()))
partidas_ids_2015_parquet = partidas_ids_2015_parquet.withColumn("Rodada", partidas_ids_2015_parquet["Rodada"].cast(IntegerType()))
partidas_ids_2015_parquet = partidas_ids_2015_parquet.withColumn("CasaID", partidas_ids_2015_parquet["CasaID"].cast(IntegerType()))
partidas_ids_2015_parquet = partidas_ids_2015_parquet.withColumn("VisitanteID", partidas_ids_2015_parquet["VisitanteID"].cast(IntegerType()))
partidas_ids_2015_parquet = partidas_ids_2015_parquet.withColumn("PlacarCasa", partidas_ids_2015_parquet["PlacarCasa"].cast(IntegerType()))
partidas_ids_2015_parquet = partidas_ids_2015_parquet.withColumn("PlacarVisitante", partidas_ids_2015_parquet["PlacarVisitante"].cast(IntegerType()))

In [116]:
partidas_ids_2015_parquet.show()

print(partidas_ids_2015_parquet.count())

+------+------+------+-----------+----------+---------------+---------+----+
|    ID|Rodada|CasaID|VisitanteID|PlacarCasa|PlacarVisitante|Resultado| ano|
+------+------+------+-----------+----------+---------------+---------+----+
|179872|     1|   262|        290|         0|              0|   Empate|2014|
|179873|     1|   266|        316|         3|              0|     Casa|2014|
|179874|     1|   276|        263|         3|              0|     Casa|2014|
|179875|     1|   277|        292|         1|              1|   Empate|2014|
|179876|     1|   293|        284|         1|              0|     Casa|2014|
|179877|     1|   282|        264|         0|              0|   Empate|2014|
|179878|     1|   265|        283|         1|              2|Visitante|2014|
|179879|     1|   285|        287|         1|              0|     Casa|2014|
|179880|     1|   288|        275|         1|              2|Visitante|2014|
|179881|     1|   315|        294|         0|              0|   Empate|2014|

## scouts_raw

In [136]:
scouts_raw_2015_df = spark.read.option("encoding", "UTF-8").csv("/cartola/data/2015/2015_scouts_raw.csv", header=True)

In [137]:
scouts_raw_2015_ano = scouts_raw_2015_df.withColumn('ano', lit(2015))

In [138]:
scouts_raw_2015_ano.show()

+------+-------+--------+-----+------+-----------+-----+-------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|Rodada|ClubeID|AtletaID|Jogos|Pontos|PontosMedia|Preco|PrecoVariacao| FS| PE|  A| FT| FD| FF|  G|  I| PP| RB| FC| GC| CA| CV| SG| DD| DP| GS| ano|
+------+-------+--------+-----+------+-----------+-----+-------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|     0|    262|   81219|    0|     0|          0|    1|            0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|2015|
|     0|    262|   88072|    0|     0|          0|    1|            0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|2015|
|     0|    262|   89258|    0|     0|          0|    1|            0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|2015|
|     0|    262|   91263|    0|     0|          0|    1|            0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  

In [139]:
# Removendo colunas
scouts_raw_2015_ano = scouts_raw_2015_ano.drop('PrecoVariacao', 'Mando', 'Titular', 'Substituido', 'TempoJogado', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS')

In [148]:
# remove todas as linhas com rodada == 0
scouts_raw_2015_ano_clean = scouts_raw_2015_ano[scouts_raw_2015_ano['Rodada'] > 0]

In [149]:
scouts_raw_2015_ano_clean.toPandas()

,Rodada,ClubeID,AtletaID,Jogos,Pontos,PontosMedia,Preco,ano
0,1,262,81121,1,-2.3,-2.3,0.79,2015
1,1,262,81219,0,0,0,1,2015
2,1,262,88072,0,0,0,1,2015
3,1,262,89258,0,0,0,1,2015
4,1,262,91263,0,0,0,1,2015
...,...,...,...,...,...,...,...,...
29715,37,317,68910,29,-0.6,3.08,8.16,2015
29716,37,317,41859,29,0,2.41,8.29,2015
29717,37,317,73528,34,0,2.06,8.52,2015
29718,37,317,37187,23,0.48,2.82,10.4,2015


In [150]:
scouts_raw_2015_ano_clean.write.parquet('/cartola/clean/scouts_raw/2015')

In [151]:
scouts_raw_2015_parquet = spark.read.parquet('/cartola/clean/scouts_raw/*')

In [152]:
scouts_raw_2015_parquet.schema
scouts_raw_2015_parquet.toPandas()

,AtletaID,Rodada,ClubeID,Pontos,PontosMedia,Preco,ano
0,81121,1,262,-2.3,-2.3,0.79,2015
1,81219,1,262,0,0,1,2015
2,88072,1,262,0,0,1,2015
3,89258,1,262,0,0,1,2015
4,91263,1,262,0,0,1,2015
...,...,...,...,...,...,...,...
60509,89924,38,None,0,-0.8,0.81,2014
60510,84794,38,293,0.9,0.9,1.24,2014
60511,89815,38,285,1,0.25,1,2014
60512,82639,38,284,0,0,1,2014


## Times

In [97]:
times_2015_df = spark.read.csv("/cartola/data/2015/2015_times.csv", header=True)

In [98]:
sorted_times_2015_df = times_2015_df.sort(times_2015_df.ID.asc())

In [99]:
times_2015_ano = times_2015_df.withColumn('ano', lit(2015))
times_2015_ano.toPandas()

,ID,Nome,Abreviacao,Slug,ano
0,262,Flamengo,fla,flamengo,2015
1,264,Corinthians,cor,corinthians,2015
2,266,Fluminense,flu,fluminense,2015
3,267,Vasco,vas,vasco,2015
4,275,Palmeiras,pal,palmeiras,2015
5,276,São Paulo,sao,sao-paulo,2015
6,277,Santos,san,santos,2015
7,282,Atlético-MG,cam,atletico-mg,2015
8,283,Cruzeiro,cru,cruzeiro,2015
9,284,Grêmio,gre,gremio,2015


In [100]:
times_2015_ano.write.parquet('/cartola/clean/times/2015')

In [101]:
times_2015_parquet = spark.read.parquet('/cartola/clean/times/*')

In [102]:
times_2015_parquet.show(100)
times_2015_parquet.schema

#Alterar ID para int.(está string)

+---+-------------+----------+-------------+----+
| ID|         Nome|Abreviacao|         Slug| ano|
+---+-------------+----------+-------------+----+
|262|     flamengo|       FLA|     flamengo|2014|
|263|     botafogo|       BOT|     botafogo|2014|
|264|  corinthians|       COR|  corinthians|2014|
|265|        bahia|       BAH|        bahia|2014|
|266|   fluminense|       FLU|   fluminense|2014|
|275|    palmeiras|       PAL|    palmeiras|2014|
|276|    são paulo|       SAO|    sao-paulo|2014|
|277|       santos|       SAN|       santos|2014|
|282|  atlético-mg|       CAM|  atletico-mg|2014|
|283|     cruzeiro|       CRU|     cruzeiro|2014|
|284|       grêmio|       GRE|       gremio|2014|
|285|internacional|       INT|internacional|2014|
|287|      vitória|       VIT|      vitoria|2014|
|288|     criciúma|       CRI|     criciuma|2014|
|290|        goiás|       GOI|        goias|2014|
|292|        sport|       SPO|        sport|2014|
|293|  atlético-pr|       CAP|  atletico-pr|2014|


StructType(List(StructField(ID,StringType,true),StructField(Nome,StringType,true),StructField(Abreviacao,StringType,true),StructField(Slug,StringType,true),StructField(ano,IntegerType,true)))

In [103]:
sorted_times = times_2015_parquet.sort(times_2015_parquet.ID.asc())

In [104]:
sorted_times.show(100)

+---+-------------+----------+-------------+----+
| ID|         Nome|Abreviacao|         Slug| ano|
+---+-------------+----------+-------------+----+
|262|     flamengo|       FLA|     flamengo|2014|
|262|     Flamengo|       fla|     flamengo|2015|
|263|     botafogo|       BOT|     botafogo|2014|
|264|  corinthians|       COR|  corinthians|2014|
|264|  Corinthians|       cor|  corinthians|2015|
|265|        bahia|       BAH|        bahia|2014|
|266|   fluminense|       FLU|   fluminense|2014|
|266|   Fluminense|       flu|   fluminense|2015|
|267|        Vasco|       vas|        vasco|2015|
|275|    palmeiras|       PAL|    palmeiras|2014|
|275|    Palmeiras|       pal|    palmeiras|2015|
|276|    são paulo|       SAO|    sao-paulo|2014|
|276|    São Paulo|       sao|    sao-paulo|2015|
|277|       Santos|       san|       santos|2015|
|277|       santos|       SAN|       santos|2014|
|282|  atlético-mg|       CAM|  atletico-mg|2014|
|282|  Atlético-MG|       cam|  atletico-mg|2015|


In [105]:
times_2015_parquet = times_2015_parquet.withColumn("ID", times_2015_parquet["ID"].cast(IntegerType()))

In [106]:
times_2015_parquet.schema

StructType(List(StructField(ID,IntegerType,true),StructField(Nome,StringType,true),StructField(Abreviacao,StringType,true),StructField(Slug,StringType,true),StructField(ano,IntegerType,true)))

In [107]:
times_2015_parquet.toPandas()

print(times_2015_parquet.count())

40


## DEMONSTRAÇÃO

In [ ]:
partidas_2014_df = spark.read.csv("/cartola/data/2014/2014_jogadores.csv", header=True)
partidas_2014_ct = partidas_2014_df.withColumn('time', regexp_replace('home_team', ' - RJ', ''))
final_partidas = partidas_2014_ct.withColumn('time_low', lower(col('time'))).show(truncate=False)

In [ ]:
with_ano_partidas = partidas_2014_ct.withColumn('ano', lit(2014)).show(truncate=False)
with_ano_partidas.show()